## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.4840,-69.3632,8.82,88,97,1.28,overcast clouds
1,1,Busselton,AU,-33.6500,115.3333,63.21,77,7,14.43,clear sky
2,2,Necochea,AR,-38.5473,-58.7368,76.32,41,0,15.08,clear sky
3,3,Tuatapere,NZ,-46.1333,167.6833,57.00,94,100,1.97,overcast clouds
4,4,Port Elizabeth,ZA,-33.9180,25.5701,84.09,51,45,11.54,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)      

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Necochea,AR,-38.5473,-58.7368,76.32,41,0,15.08,clear sky
4,4,Port Elizabeth,ZA,-33.9180,25.5701,84.09,51,45,11.54,scattered clouds
6,6,Port Alfred,ZA,-33.5906,26.8910,88.18,40,95,1.99,overcast clouds
9,9,Pondicherry,IN,11.9300,79.8300,82.36,82,0,11.65,clear sky
10,10,Hambantota,LK,6.1241,81.1185,79.14,100,40,2.30,scattered clouds
12,12,Atuona,PF,-9.8000,-139.0333,78.82,77,88,16.17,overcast clouds
13,13,Rikitea,PF,-23.1203,-134.9692,77.56,72,54,4.72,broken clouds
21,21,Isangel,VU,-19.5500,169.2667,77.70,86,70,7.09,broken clouds
25,25,Bredasdorp,ZA,-34.5322,20.0403,75.85,39,72,9.84,broken clouds
30,30,Puerto Ayora,EC,-0.7393,-90.3518,84.16,72,88,10.74,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                222
City                   222
Country                220
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Current Description    222
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Necochea,AR,76.32,clear sky,-38.5473,-58.7368,
4,Port Elizabeth,ZA,84.09,scattered clouds,-33.9180,25.5701,
6,Port Alfred,ZA,88.18,overcast clouds,-33.5906,26.8910,
9,Pondicherry,IN,82.36,clear sky,11.9300,79.8300,
10,Hambantota,LK,79.14,scattered clouds,6.1241,81.1185,
12,Atuona,PF,78.82,overcast clouds,-9.8000,-139.0333,
13,Rikitea,PF,77.56,broken clouds,-23.1203,-134.9692,
21,Isangel,VU,77.70,broken clouds,-19.5500,169.2667,
25,Bredasdorp,ZA,75.85,broken clouds,-34.5322,20.0403,
30,Puerto Ayora,EC,84.16,overcast clouds,-0.7393,-90.3518,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# 6b. Iterate through the hotel DataFrame.# 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Necochea,AR,76.32,clear sky,-38.5473,-58.7368,dyd -mar Hotel
4,Port Elizabeth,ZA,84.09,scattered clouds,-33.9180,25.5701,39 On Nile Guest House
6,Port Alfred,ZA,88.18,overcast clouds,-33.5906,26.8910,The Halyards Hotel
9,Pondicherry,IN,82.36,clear sky,11.9300,79.8300,Villa Shanti for food lovers
10,Hambantota,LK,79.14,scattered clouds,6.1241,81.1185,Bungalow 63
12,Atuona,PF,78.82,overcast clouds,-9.8000,-139.0333,Villa Enata
13,Rikitea,PF,77.56,broken clouds,-23.1203,-134.9692,People ThankYou
21,Isangel,VU,77.70,broken clouds,-19.5500,169.2667,Tanna Lodge
25,Bredasdorp,ZA,75.85,broken clouds,-34.5322,20.0403,Bredasdorp Country Manor
30,Puerto Ayora,EC,84.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df=hotel_df[hotel_df["Hotel Name"]!= '']
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Necochea,AR,76.32,clear sky,-38.5473,-58.7368,dyd -mar Hotel
4,Port Elizabeth,ZA,84.09,scattered clouds,-33.9180,25.5701,39 On Nile Guest House
6,Port Alfred,ZA,88.18,overcast clouds,-33.5906,26.8910,The Halyards Hotel
9,Pondicherry,IN,82.36,clear sky,11.9300,79.8300,Villa Shanti for food lovers
10,Hambantota,LK,79.14,scattered clouds,6.1241,81.1185,Bungalow 63
...,...,...,...,...,...,...,...
625,Sao Miguel Do Araguaia,BR,87.67,broken clouds,-13.2750,-50.1628,Hotel Executivo Palace
631,Manta,EC,85.78,scattered clouds,-0.9500,-80.7333,Balandra Hotel
632,Kupang,ID,77.77,broken clouds,-10.1667,123.5833,Hotel On The Rock by Prasanthi
635,Antalaha,MG,77.76,few clouds,-14.9003,50.2788,Hotel Vitasoa


In [11]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description/dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))